D'abord : se logguer à un compte gmail \
login : eleveenmi@gmail.com  (i=1,2,3,..) \
mwd : pwdeleveEnmi (i=1,2,3,..)

# I) Installations et imports

In [ ]:
# Installation et imports
!pip install --upgrade gspread gspread_dataframe
from google.colab import auth
from google.auth import default
import gspread
from google.colab import drive
from google.colab import output
import os
import re
import glob
import shutil
import tarfile
from pathlib import Path
import random
from IPython.display import display, clear_output, Javascript
import ipywidgets as widgets
from base64 import b64encode
from datetime import datetime, timedelta
import pandas as pd
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import re

# Spécifique au TP :
! git clone https://github.com/nanopiero/intro-ia-secondes-2025.git
! cp intro-ia-secondes-2025/utils/annotation.py .
! cp intro-ia-secondes-2025/utils/configuration.py .
# from google.colab import files
# uploaded = files.upload()
from importlib import reload
import configuration as cf
import annotation as an
# reload(cf)
# reload(an)

# II) Authentification

In [ ]:
# pour colab :
auth.authenticate_user()
# pour accéder aux fichiers google (google sheet)
creds, _ = default()  # or use service account
gc = gspread.authorize(creds)

# montage du drive :
drive.mount('/content/drive')

# III) Configuration

In [ ]:
groupe = 1
direction, starthour = cf.group2direction_and_starthour[groupe]
image_dir = f'{groupe}_{direction}_{starthour}'
print(image_dir)

# IV) Récupération des images :

In [6]:
# 1. Define the source path (change SHARED_PATH accordingly)
tar_and_zipped_image_dir = image_dir + '.tar.gz'
shared_path = '/content/drive/MyDrive/stage_secondes_2025/' + tar_and_zipped_image_dir
dest_path = Path.home() / tar_and_zipped_image_dir

# Copy the file to your home directory
shutil.copy(shared_path, dest_path)

# 2. Extract and delete the archive
with tarfile.open(dest_path, 'r:gz') as tar:
    tar.extractall(path='.')  # extract to current working directory

# 3. Remove the tar.gz file
dest_path.unlink()

# V) Récupération du fichier google (et des labels qu'il peut contenir)

In [ ]:
# Open by ID or URL
sheet_url = cf.group2sheets[groupe]
sh = gc.open_by_url(sheet_url)
worksheet = sh.sheet1
print(worksheet.url)

# VI) Annotation

In [ ]:
ui = an.AnnotationUI(image_dir, worksheet, direction)